In [1]:
import cv2
import os

# Video file path
video_path = '/home/kyoungsu/Desktop/20241220_135359.mp4'
output_folder = '/home/kyoungsu/Download/output/'
os.makedirs(output_folder, exist_ok=True)

# Open the video file
cap = cv2.VideoCapture(video_path)

# Get total number of frames
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

# Define the step to extract approximately 40 frames
num_frames_to_extract = 40
step = max(total_frames // num_frames_to_extract, 1)

frame_count = 0
saved_count = 0

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Save every 'step' frame
    if frame_count % step == 0:
        output_path = os.path.join(output_folder, f'frame_{saved_count:04d}.jpg')
        cv2.imwrite(output_path, frame)
        saved_count += 1

    frame_count += 1

    # Stop if we have saved enough frames
    if saved_count >= num_frames_to_extract:
        break

cap.release()
print(f"Saved {saved_count} frames to {output_folder}")


Saved 40 frames to /home/kyoungsu/Download/output/


In [3]:
import os
from moviepy.video.io.VideoFileClip import VideoFileClip

# Video file path
video_path = '/home/kyoungsu/Desktop/20241220_135359.mp4'
output_folder = '/home/kyoungsu/Downloads/output/'
os.makedirs(output_folder, exist_ok=True)

# Load the video file using MoviePy
clip = VideoFileClip(video_path)

# Get total number of frames based on video duration and fps
total_frames = int(clip.fps * clip.duration)

# Define the step to extract approximately 40 frames
num_frames_to_extract = 40
step = max(total_frames // num_frames_to_extract, 1)

frame_count = 0
saved_count = 0

for frame in clip.iter_frames(fps=clip.fps, dtype="uint8"):
    if frame_count % step == 0:
        output_path = os.path.join(output_folder, f'frame_{saved_count:04d}.jpg')
        # Save the frame using MoviePy's PIL integration
        from PIL import Image
        img = Image.fromarray(frame)
        img.save(output_path)
        saved_count += 1

    frame_count += 1

    # Stop if we have saved enough frames
    if saved_count >= num_frames_to_extract:
        break

clip.close()
print(f"Saved {saved_count} frames to {output_folder}")


Saved 40 frames to /home/kyoungsu/Downloads/output/


In [4]:
import os
from moviepy.video.io.VideoFileClip import VideoFileClip
import numpy as np
from PIL import Image, ImageDraw

# Helper function for camera calibration
def calibrate_camera(chessboard_images, pattern_size):
    import cv2

    obj_points = []
    img_points = []
    objp = np.zeros((pattern_size[0] * pattern_size[1], 3), np.float32)
    objp[:, :2] = np.mgrid[0:pattern_size[0], 0:pattern_size[1]].T.reshape(-1, 2)

    for img_path in chessboard_images:
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        ret, corners = cv2.findChessboardCorners(img, pattern_size, None)
        if ret:
            img_points.append(corners)
            obj_points.append(objp)

    ret, K, D, rvecs, tvecs = cv2.calibrateCamera(obj_points, img_points, img.shape[::-1], None, None)
    return K, D

# Define pattern size for calibration
pattern_size = (9, 6)  # Example chessboard pattern size

# Video file path
video_path = '/home/kyoungsu/Desktop/20241220_135359.mp4'
output_folder = '/home/kyoungsu/Downloads/output/'
os.makedirs(output_folder, exist_ok=True)

# Load the video file using MoviePy
clip = VideoFileClip(video_path)

# Get total number of frames based on video duration and fps
total_frames = int(clip.fps * clip.duration)

# Define the step to extract approximately 40 frames
num_frames_to_extract = 40
step = max(total_frames // num_frames_to_extract, 1)

frame_count = 0
saved_count = 0

chessboard_images = []

for frame in clip.iter_frames(fps=clip.fps, dtype="uint8"):
    if frame_count % step == 0:
        output_path = os.path.join(output_folder, f'frame_{saved_count:04d}.jpg')
        # Save the frame using MoviePy's PIL integration
        img = Image.fromarray(frame)
        img.save(output_path)
        chessboard_images.append(output_path)
        saved_count += 1

    frame_count += 1

    # Stop if we have saved enough frames
    if saved_count >= num_frames_to_extract:
        break

clip.close()
print(f"Saved {saved_count} frames to {output_folder}")

# Calibrate the camera
K, D = calibrate_camera(chessboard_images, pattern_size)

# Draw a cube on one of the calibrated images
def draw_cube_on_image(img_path, K, D, pattern_size):
    import cv2

    img = cv2.imread(img_path)
    ret, corners = cv2.findChessboardCorners(cv2.cvtColor(img, cv2.COLOR_BGR2GRAY), pattern_size, None)

    if not ret:
        print(f"Chessboard not found in {img_path}")
        return img

    # Define 3D cube points
    square_size = 0.025  # Adjust based on your calibration square size
    cube_points = np.float32([
        [0, 0, 0], [0, square_size, 0], [square_size, square_size, 0], [square_size, 0, 0],
        [0, 0, -square_size], [0, square_size, -square_size], [square_size, square_size, -square_size], [square_size, 0, -square_size]
    ])

    # Project 3D points to 2D image points
    ret, rvecs, tvecs = cv2.solvePnP(obj_points[0], corners, K, D)
    imgpts, _ = cv2.projectPoints(cube_points, rvecs, tvecs, K, D)
    imgpts = np.int32(imgpts).reshape(-1, 2)

    # Draw cube edges
    img = cv2.drawContours(img, [imgpts[:4]], -1, (0, 255, 0), 3)  # Base
    for i, j in zip(range(4), range(4, 8)):
        img = cv2.line(img, tuple(imgpts[i]), tuple(imgpts[j]), (255, 0, 0), 3)  # Vertical edges
    img = cv2.drawContours(img, [imgpts[4:]], -1, (0, 0, 255), 3)  # Top

    return img

# Example usage
target_image = chessboard_images[0]
output_image = draw_cube_on_image(target_image, K, D, pattern_size)
cv2.imwrite('/home/kyoungsu/Downloads/output/frame_0000.jpg', output_image)
print("Cube drawn on image and saved as output_with_cube.jpg")


/home/kyoungsu/anaconda3/envs/pa2/lib/python3.9/site-packages/moviepy/video/io/ffmpeg_reader.py:157: UserWarning: In file path_to_your_video.mp4, 3 bytes wanted but 0 bytes read at frame index 0 (out of a total 0 frames), at time 0.00/0.00 sec. Using the last valid frame instead.
  warnings.warn(


OSError: MoviePy error: failed to read the first frame of video file path_to_your_video.mp4. That might mean that the file is corrupted. That may also mean that you are using a deprecated version of FFMPEG. On Ubuntu/Debian for instance the version in the repos is deprecated. Please update to a recent version from the website.